In [ ]:
pip install wget

In [121]:
import astropy
from astropy.table import Table,vstack, Column
from subprocess import call
import subprocess
from astropy.io import fits
import numpy as np
import wget
import matplotlib.pyplot as plt
from gpz_pype.utilities import Params, set_gpz_path, basic_lupt_soft, flux_to_lupt
import astropy.units as u
from numpy.ma import masked
from astropy.coordinates import SkyCoord
from astropy.coordinates import match_coordinates_sky

In [ ]:
fields= ['ceers-full','gdn','gds','ngdeep','primer-cosmos-east','primer-cosmos-west','primer-uds-north','primer-uds-south','abell2744clu','abells1063','elgordo','j112716p4228','macs0647','rxj2129','smacs0723','sunrise']

In [ ]:
for field in fields:
    url=f'https://s3.amazonaws.com/grizli-v2/JwstMosaics/v7/{field}-grizli-v7.0_phot.fits'
    filename=wget.download(url)
    print(f' {field} downloaded')

In [ ]:
###INITIAL CATALOGUE MAKING
hdu_list = fits.open('MPhys/DAWN_JWST/ceers-full-grizli-v7.0_phot.fits', memmap=True,formats='fits')
ceers = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/gdn-grizli-v7.0_phot.fits', memmap=True,formats='fits')
gdn= Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/gds-grizli-v7.0_phot.fits', memmap=True,formats='fits')
gds = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/ngdeep-grizli-v7.0_phot.fits', memmap=True,formats='fits')
ngdeep = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/primer-cosmos-east-grizli-v7.0_phot.fits', memmap=True,formats='fits')
cosmos_east = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/primer-cosmos-west-grizli-v7.0_phot.fits', memmap=True,formats='fits')
cosmos_west = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/primer-uds-south-grizli-v7.0_phot.fits', memmap=True,formats='fits')
udss = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/primer-uds-north-grizli-v7.0_phot.fits', memmap=True,formats='fits')
udsn = Table(hdu_list[1].data)
hdu_list.close()

In [ ]:
#ADDING FIELDS TO CATALOGUE - RUN BOTH CELLS IF STARTING FROM SCRATCH
hdu_list = fits.open('MPhys/DAWN_JWST/abell2744clu-grizli-v7.0_phot.fits', memmap=True,formats='fits')
abell2744 = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/abells1063-grizli-v7.0_phot.fits', memmap=True,formats='fits')
abells1063= Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/elgordo-grizli-v7.0_phot.fits', memmap=True,formats='fits')
elgordo = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/j112716p4228-grizli-v7.0_phot.fits', memmap=True,formats='fits')
j_field = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/macs0647-grizli-v7.0_phot.fits', memmap=True,formats='fits')
macs = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/rxj2129-grizli-v7.0_phot.fits', memmap=True,formats='fits')
rxj = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/smacs0723-grizli-v7.0_phot.fits', memmap=True,formats='fits')
smacs = Table(hdu_list[1].data)
hdu_list.close()

hdu_list = fits.open('MPhys/DAWN_JWST/sunrise-grizli-v7.0_phot.fits', memmap=True,formats='fits')
sunrise = Table(hdu_list[1].data)
hdu_list.close()

In [ ]:
abell2744.add_column('abells2744clu', name='field', index=0)
abells1063.add_column('abells1063', name='field', index=0)
elgordo.add_column('elgordo', name='field', index=0)
j_field.add_column('j112716p4228', name='field', index=0)
macs.add_column('macs0647', name='field', index=0)
rxj.add_column('rxj2129', name='field', index=0)
smacs.add_column('smacs0723', name='field', index=0)
sunrise.add_column('sunrise', name='field', index=0)

In [ ]:
ceers.add_column('ceers', name='field', index=0)
gdn.add_column('gdn', name='field', index=0)
gds.add_column('gds', name='field', index=0)
ngdeep.add_column('ngdeep', name='field', index=0)
cosmos_east.add_column('cosmos_east', name='field', index=0)
cosmos_west.add_column('cosmos_west', name='field', index=0)
udsn.add_column('udsn', name='field', index=0)
udss.add_column('udss', name='field', index=0)

In [ ]:
fields_new= ['ceers','gdn','gds','ngdeep','cosmos_east','cosmos_west','udsn','udss','abell2744','abells1063','elgordo','j_field','macs','rxj','smacs','sunrise']

In [ ]:
field=sunrise


In [ ]:
filters0 = [col.split('_')[0] for col in field.colnames if col.endswith('flux_aper_0')]
filters0=filters0[1:]
filters1 = [col.split('_')[0] for col in field.colnames if col.endswith('flux_aper_1')]
filters1=filters1[1:]

filters2 = [col.split('_')[0] for col in field.colnames if col.endswith('flux_aper_2')]
filters2=filters2[1:]

filters3 = [col.split('_')[0] for col in field.colnames if col.endswith('flux_aper_3')]
filters3=filters3[1:]



In [ ]:

b_arr0 = [basic_lupt_soft(field[f'{filt}_flux_aper_0'], field[f'{filt}_fluxerr_aper_0']) for filt in filters0] 
b_arr1 = [basic_lupt_soft(field[f'{filt}_flux_aper_1'], field[f'{filt}_fluxerr_aper_1']) for filt in filters1] 
b_arr2 = [basic_lupt_soft(field[f'{filt}_flux_aper_2'], field[f'{filt}_fluxerr_aper_2']) for filt in filters2] 
b_arr3 = [basic_lupt_soft(field[f'{filt}_flux_aper_3'], field[f'{filt}_fluxerr_aper_3']) for filt in filters3] 

In [ ]:
i=15
lupt_cols=field['field','id','ra','dec']

for filt,b in zip(filters0,b_arr0):
    lupt,lupterr = flux_to_lupt(field[f'{filt}_flux_aper_0'],field[f'{filt}_fluxerr_aper_0'],b)
    lupt_cols[f'lupt_{filt}_aper0']=lupt
    lupt_cols[f'lupterr_{filt}_aper0']=lupterr
for filt,b in zip(filters1,b_arr1):
    lupt,lupterr = flux_to_lupt(field[f'{filt}_flux_aper_1'],field[f'{filt}_fluxerr_aper_1'],b)
    lupt_cols[f'lupt_{filt}_aper1']=lupt
    lupt_cols[f'lupterr_{filt}_aper1']=lupterr
for filt,b in zip(filters2,b_arr2):
    lupt,lupterr = flux_to_lupt(field[f'{filt}_flux_aper_2'],field[f'{filt}_fluxerr_aper_2'],b)
    lupt_cols[f'lupt_{filt}_aper2']=lupt
    lupt_cols[f'lupterr_{filt}_aper2']=lupterr
for filt,b in zip(filters3,b_arr3):
    lupt,lupterr = flux_to_lupt(field[f'{filt}_flux_aper_3'],field[f'{filt}_fluxerr_aper_3'],b)
    lupt_cols[f'lupt_{filt}_aper3']=lupt
    lupt_cols[f'lupterr_{filt}_aper3']=lupterr

lupt_cols.write(f'MPhys/catalogues/{fields[i]}_lupts.fits',format='fits', overwrite=True)

In [2]:
lup_ceers=Table.read('MPhys/catalogues/ceers-full_lupts.fits',format='fits')
lup_gds=Table.read('MPhys/catalogues/gds_lupts.fits',format='fits')
lup_gdn=Table.read('MPhys/catalogues/gdn_lupts.fits',format='fits')
lup_ngdeep=Table.read('MPhys/catalogues/ngdeep_lupts.fits',format='fits')
lup_udss=Table.read('MPhys/catalogues/primer-uds-south_lupts.fits',format='fits')
lup_udsn=Table.read('MPhys/catalogues/primer-uds-north_lupts.fits',format='fits')
lup_cosmos_east=Table.read('MPhys/catalogues/primer-cosmos-east_lupts.fits',format='fits')
lup_cosmos_west=Table.read('MPhys/catalogues/primer-cosmos-west_lupts.fits',format='fits')

In [3]:
lup_abell2744clu = Table.read('MPhys/catalogues/abell2744clu_lupts.fits',format='fits')
lup_abells1063=Table.read('MPhys/catalogues/abells1063_lupts.fits',format='fits')
lup_elgordo=Table.read('MPhys/catalogues/elgordo_lupts.fits',format='fits')
lup_j112716p4228=Table.read('MPhys/catalogues/j112716p4228_lupts.fits',format='fits')
lup_macs0647=Table.read('MPhys/catalogues/macs0647_lupts.fits',format='fits')
lup_rxj2129=Table.read('MPhys/catalogues/rxj2129_lupts.fits',format='fits')
lup_smacs0723=Table.read('MPhys/catalogues/smacs0723_lupts.fits',format='fits')
lup_sunrise=Table.read('MPhys/catalogues/sunrise_lupts.fits',format='fits')

In [4]:

lup_con =vstack([lup_ceers,lup_gds,lup_gdn,lup_ngdeep,lup_udss,lup_udsn,lup_cosmos_east,lup_cosmos_west,])


In [5]:
lup_con_small = vstack([lup_abell2744clu,lup_abells1063,lup_elgordo,lup_j112716p4228,lup_macs0647,lup_rxj2129,lup_smacs0723,lup_sunrise])

In [6]:
lup_total = vstack([lup_con,lup_con_small])

In [ ]:
#needs to be done in CP Lab 
#lup_total.write('luptitudes_all_jwst.fits', format='fits', overwrite=True)

In [39]:
#Full catalogue filter names 
filters_1 = [col.split('_')[1] for col in lup_total.colnames if col.endswith('_aper0')]
#print(len(filters_1))
x= int(184/4)+4
#lup_total['lupt_f250m-clear_aper0']
print(filters_1)

['f1000w', 'f1000w', 'f105w', 'f105w', 'f115w-clear', 'f115w-clear', 'f125w', 'f125w', 'f1280w', 'f1280w', 'f140w', 'f140w', 'f1500w', 'f1500w', 'f150w-clear', 'f150w-clear', 'f160w', 'f160w', 'f1800w', 'f1800w', 'f182m-clear', 'f182m-clear', 'f200w-clear', 'f200w-clear', 'f2100w', 'f2100w', 'f210m-clear', 'f210m-clear', 'f275wu', 'f275wu', 'f277w-clear', 'f277w-clear', 'f356w-clear', 'f356w-clear', 'f410m-clear', 'f410m-clear', 'f435w', 'f435w', 'f444w-clear', 'f444w-clear', 'f560w', 'f560w', 'f606w', 'f606w', 'f606wu', 'f606wu', 'f770w', 'f770w', 'f814w', 'f814w', 'f090w-clear', 'f090w-clear', 'f335m-clear', 'f335m-clear', 'f350lpu', 'f350lpu', 'f475w', 'f475w', 'f775w', 'f775w', 'f850lp', 'f850lp', 'f110w', 'f110w', 'f336wu', 'f336wu', 'f850lpu', 'f850lpu', 'clearp-f356w', 'clearp-f356w', 'clearp-f444w', 'clearp-f444w', 'f115wn-clear', 'f115wn-clear', 'f150wn-clear', 'f150wn-clear', 'f200wn-clear', 'f200wn-clear', 'f225wu', 'f225wu', 'f250m-clear', 'f250m-clear', 'f300m-clear', 'f30

In [57]:
#### FILTER CHOICES
c1 =['f625w','f110w','f277w-clear','f356w-clear','f444w-clear']
c2 =['f606w','f150w-clear','f250m-clear','f300m-clear','f444w-clear']
c3 =['f814w','f110w','f277w-clear','f435w','f480m-clear']
c4 =['f555w','f140w','f200w-clear','f356w-clear','f480m-clear']
c5 =['f606w','f110w','f200w-clear','f356w-clear','f444w-clear']
print(c1[0])

f625w


In [80]:
######change this to make new catalogue of filters
aper=0
choice=c5
lup_train = lup_total['field','id','ra','dec']
lup_train[f'lupt_{choice[0]}']=lup_total[f'lupt_{choice[0]}_aper{aper}']
lup_train[f'lupterr_{choice[0]}']=lup_total[f'lupterr_{choice[0]}_aper{aper}']
lup_train[f'lupt_{choice[1]}']=lup_total[f'lupt_{choice[1]}_aper{aper}']
lup_train[f'lupterr_{choice[1]}']=lup_total[f'lupterr_{choice[1]}_aper{aper}']
lup_train[f'lupt_{choice[2]}']=lup_total[f'lupt_{choice[2]}_aper{aper}']
lup_train[f'lupterr_{choice[2]}']=lup_total[f'lupterr_{choice[2]}_aper{aper}']
lup_train[f'lupt_{choice[3]}']=lup_total[f'lupt_{choice[3]}_aper{aper}']
lup_train[f'lupterr_{choice[3]}']=lup_total[f'lupterr_{choice[3]}_aper{aper}']
lup_train[f'lupt_{choice[4]}']=lup_total[f'lupt_{choice[4]}_aper{aper}']
lup_train[f'lupterr_{choice[4]}']=lup_total[f'lupterr_{choice[4]}_aper{aper}']

In [76]:
lup_train

field,id,ra,dec,lupt_f606w,lupterr_f606w,lupt_f110w,lupterr_f110w,lupt_f200w-clear,lupterr_f200w-clear,lupt_f356w-clear,lupterr_f356w-clear,lupt_f444w-clear,lupterr_f444w-clear
bytes13,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
ceers,1,215.2151731300751,52.97528909359392,29.556780901764032,0.5946342561731784,--,--,--,--,--,--,--,--
ceers,2,215.1865311684452,52.95489247370195,28.744782309160975,0.3691622925442938,--,--,--,--,--,--,--,--
ceers,3,215.17440399645494,52.94624855014534,30.5359365634207,0.5906980751137015,--,--,--,--,--,--,--,--
ceers,4,215.17066593131037,52.9435832117646,30.33628891980326,0.624851440928911,--,--,--,--,--,--,--,--
ceers,5,215.1665950611378,52.940680067642184,30.039431762503852,0.6493763665679093,--,--,--,--,--,--,--,--
ceers,6,215.1651987046283,52.939684133167596,28.880511830952422,0.4311701103603547,--,--,--,--,--,--,--,--
ceers,7,215.1458629783668,52.92588676172599,25.04257048559836,0.05662983390736154,--,--,--,--,--,--,--,--
ceers,8,215.13093623809937,52.91522733871391,30.98113463283675,0.46693214149978884,--,--,--,--,--,--,--,--
ceers,9,215.12435951625753,52.91052853715476,30.0292321491326,0.6861669860612213,--,--,--,--,--,--,--,--


In [ ]:
#filts=['f115w','f200w','f277w','f356w','f444w']

In [81]:
f1=[]
for i in range(len(lup_train)):
    f1.append(np.ma.is_masked(lup_train[f'lupt_{choice[0]}'][i]))
f1_inv=np.array(np.invert(f1))
f1_cat =lup_train[f1_inv*(np.ma.is_masked(lup_train[f'lupt_{choice[0]}']) == True)] 

In [82]:

f2=[]
for i in range(len(f1_cat)):
    f2.append(np.ma.is_masked(f1_cat[f'lupt_{choice[1]}'][i]))
f2_inv=np.array(np.invert(f2))
f2_cat =f1_cat[f2_inv*(np.ma.is_masked(f1_cat[f'lupt_{choice[1]}']) == True)] 

In [ ]:
f2_cat

In [85]:

f3=[]
for i in range(len(f2_cat)):
    f3.append(np.ma.is_masked(f2_cat[f'lupt_{choice[2]}'][i]))
f3_inv=np.array(np.invert(f3))
f3_cat =f2_cat[f3_inv*(np.ma.is_masked(f2_cat[f'lupt_{choice[2]}']) == True)] 

In [86]:
f3_cat

field,id,ra,dec,lupt_f606w,lupterr_f606w,lupt_f110w,lupterr_f110w,lupt_f200w-clear,lupterr_f200w-clear,lupt_f356w-clear,lupterr_f356w-clear,lupt_f444w-clear,lupterr_f444w-clear
bytes13,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
abells1063,12,342.194428067702,-44.55473213169154,29.31103798376593,0.19811724354271665,33.31977486720569,0.879857034851387,27.41069293637933,0.15205261996150898,--,--,28.034983423632752,0.24600925412242872
abells1063,13,342.19277442978813,-44.554513851853876,23.407891841927814,0.0021240320785500884,22.17952746845046,0.0036481118567834067,21.295067127059948,0.0009284620094364807,--,--,22.343590781768217,0.0020091802440730805
abells1063,15,342.194762405608,-44.554199203818555,28.8821025975773,0.13550978153887686,26.487398725800755,0.18516738982211983,26.118570317984467,0.046802368208558286,--,--,25.82778216817152,0.033280768015753255
abells1063,16,342.19268402736463,-44.553991704265435,27.742986144313697,0.04929329209749163,27.220233116576495,0.357597080833786,26.101098995769252,0.04523483838292508,--,--,26.50737553117812,4.189170655978912
abells1063,18,342.19477568427067,-44.55370398741217,27.404263774736055,0.03663555970722581,26.916978408206155,0.2929644828162571,26.377052907997204,0.04087990607107707,--,--,26.575614736827937,0.06884616962440734
abells1063,21,342.18801723161243,-44.55272705170651,27.373638681377805,0.03627608305317661,26.4094235512353,0.17443170432499092,25.876519137276954,0.03635566937078513,--,--,27.247817001382234,0.1342805640626889
abells1063,22,342.188111250398,-44.55261356292642,28.209708316456595,0.07613693780880272,27.086789625313973,0.31724388360018757,26.874744416228694,0.09161278877338312,--,--,27.285047113319955,0.12647586326122315
abells1063,23,342.1938642600391,-44.552634171934095,26.549094335971624,0.017577840398386824,26.626446133577904,0.20936125614735593,25.477227299045527,0.018300179614078208,--,--,25.500549902217905,0.017891468618298895
abells1063,24,342.19414058450724,-44.55253264875912,28.878297336612583,0.1332667185091232,29.83795909945683,1.86836138289304,28.152950133357354,0.19691505048148028,--,--,27.746781095157598,0.13182462959115834


In [87]:

f4=[]
for i in range(len(f3_cat)):
    f4.append(np.ma.is_masked(f3_cat[f'lupt_{choice[3]}'][i]))
f4_inv=np.array(np.invert(f4))
f4_cat =f3_cat[f4_inv*(np.ma.is_masked(f3_cat[f'lupt_{choice[3]}']) == True)] #false idk why but works 


In [88]:
f4_cat

field,id,ra,dec,lupt_f606w,lupterr_f606w,lupt_f110w,lupterr_f110w,lupt_f200w-clear,lupterr_f200w-clear,lupt_f356w-clear,lupterr_f356w-clear,lupt_f444w-clear,lupterr_f444w-clear
bytes13,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
macs0647,3976,101.99435324482509,70.22764130455266,27.727528478359403,0.20240082640969292,27.453859514592633,0.17045431320062146,26.917512530195864,0.06365778761834598,26.93216800271955,0.057968604927670336,26.96201714167506,0.09517025119652596
macs0647,3978,101.9898333307484,70.227928956308,29.841388042135815,0.4074897376152559,28.798417282416835,0.40856640358624075,27.843588030051198,0.14908943382064704,28.215639317717695,0.17368960490570712,28.692321646441016,0.36319803633173664
macs0647,3979,101.98921818456209,70.22794548695317,29.113981170654487,0.43146188677665853,29.016459962801452,0.4531457118276954,28.970684544760857,0.38197728227525185,29.427274773881383,0.9089692675161898,29.050896895432768,0.5297540114132252
macs0647,3983,101.99716204697782,70.22779213635883,26.796835138356403,0.09352237189957867,25.53000272846882,0.028333051778609064,24.96434543803402,0.011203339125944315,24.77599818880748,0.008079993649095427,24.64854819798589,0.010683250174254278
macs0647,3984,101.99694580048542,70.22806024826556,27.56086309451508,0.22033584724007277,27.52642481252893,0.1548192286377876,27.003735857427234,0.0691283172205789,27.653995486037026,0.10160811674372949,27.829345659775033,0.193430172748305
macs0647,3985,101.98882790133804,70.22818239896256,28.36566290528209,1.2222330844407558,28.0826224200209,0.25143814775246637,27.453349405241692,0.1035326472295886,27.556702074540297,0.09170029613199109,27.569587153837556,0.1451244721712734
macs0647,3986,101.99815976931545,70.22837322847,28.187887766493063,0.4523776526239224,29.235737813396494,0.5054168798669746,27.884691229866927,0.1776266847951962,28.42695964860224,0.18710910266968664,28.21963486486782,0.250486930840804
macs0647,3987,101.98867665470219,70.22838770587835,28.75900663859935,0.49556597252706414,29.220548027497426,0.48136351868358945,28.95548901354399,0.4248759284818994,28.729614742338068,0.24223742815138571,28.37130165279834,0.2770087380176852
macs0647,3988,101.99791877956773,70.22840335205194,28.302172504803156,2.242493287398623,28.99292439680199,0.447660604554518,28.079127035529723,0.19586404148722789,28.696453265282067,0.23797712975842156,28.87424577213734,0.4158081136723979


In [89]:
f5=[]
for i in range(len(f4_cat)):
    f5.append(np.ma.is_masked(f4_cat[f'lupt_{choice[4]}'][i]))
f5_inv=np.array(np.invert(f5))
f5_cat =f4_cat[f5_inv*(np.ma.is_masked(f4_cat[f'lupt_{choice[4]}']) == True)] 

In [120]:
np.max(f5_cat['id'])

7854

In [91]:
f5_cat.write('MPhys/catalogues/filter_choice_5.fits',format = 'fits',overwrite=True)

In [98]:
candels=Table.read('MPhys/CANDELS/CANDELS_catalogue.fits',format='fits')
candels.colnames

['ID', 'RA', 'DEC', 'z_spec', 'spec_qual']

In [99]:
ra1=f5_cat['ra']
dec1=f5_cat['dec']
ra2=candels['RA']
dec2=candels['DEC']

In [128]:
c = SkyCoord(ra=ra1*u.degree, dec=dec1*u.degree)#jwst
catalog = SkyCoord(ra=ra2*u.degree, dec=dec2*u.degree)#candels
#idx, d2d, d3d = c.match_to_catalog_sky(catalog)#gives id of candels catalogue
#id_jwst, d2d_jwst, d3d_jwst = catalog.match_to_catalog_sky(c) #id of jwst catalogue for filter choice

#idx, d2d, d3d = match_coordinates_sky(c, catalog)

In [132]:
max_sep = 1.0 * u.arcsec
idx, d2d, d3d = c.match_to_catalog_sky(catalog)
sep_constraint = d2d < max_sep
c_matches = c[sep_constraint]
catalog_matches = catalog[idx[sep_constraint]]

In [135]:
catalog_matches

<SkyCoord (ICRS): (ra, dec) in deg
    []>

TypeError: 'numpy.int32' object does not support item assignment

6277

In [ ]:
fit= Table.read('MPhys/jwst_parameter_files/jwst_nans_removed_lupts_specz_TRAIN.fits', format = 'fits')
train = fit['id_1','ra_1','dec_1','lupt_f115w','lupterr_f115w','lupt_f200w','lupterr_f200w','lupt_f277w','lupterr_f277w','lupt_f356w','lupterr_f356w','lupt_f444w','lupterr_f444w','z_spec']
train.write('MPhys/gpz_files/jwst_TRAIN_2.cat',format='ascii.commented_header',overwrite=True)

In [ ]:
train.sort('id_1')

In [ ]:
train_split=train[0:int(0.8*len(train))]
pred_split=train[int(0.8*len(train)):-1]

In [ ]:
train_split.write('MPhys/gpz_files/TRAIN_80_split.cat', format ='ascii.commented_header', overwrite=True)
pred_split.write('MPhys/gpz_files/PRED_20_split.cat', format ='ascii.commented_header', overwrite=True)

In [ ]:
pred_split

In [ ]:
ceers_new.write('ceers_tst.fits',format='fits',overwrite=True)

In [ ]:
filt='f115w-clear_flux_aper_1'
cat_ceers = ceers[f'{filt}','flux_auto','flux_aper_1']
cat_gds= gds[f'{filt}','flux_auto','flux_aper_1']
cat_gdn = gdn[f'{filt}','flux_auto','flux_aper_1']
cat_ngdeep = ngdeep[f'{filt}','flux_auto','flux_aper_1']
cat_cosmos_west = cosmos_west[f'{filt}','flux_auto','flux_aper_1']
cat_cosmos_east = cosmos_east[f'{filt}','flux_auto','flux_aper_1']
cat_udss = udss[f'{filt}','flux_auto','flux_aper_1']
cat_udsn = udsn[f'{filt}','flux_auto','flux_aper_1']


    


In [ ]:
flux_ceers=[]
mag_ceers=[]

flux_gds=[]
mag_gds=[]

flux_gdn=[]
#mag_gdn=[]

flux_ngdeep=[]
mag_ngdeep=[]

flux_cosmos_west=[]
mag_cosmos_west=[]

flux_udss=[]
mag_udss=[]

flux_udsn=[]
#mag_udsn=[]

flux_cosmos_east=[]
#mag_cosmos_east=[]

for j in range(len(cat_ceers)):
    flux_ceers.append((cat_ceers[f'{filt}'][j] * cat_ceers['flux_auto'][j]) /cat_ceers['flux_aper_1'][j])
    mag_ceers.append(23.9-(2.5*np.log10(flux_ceers[j])))
for j in range(len(cat_gds)):
    flux_gds.append((cat_gds[f'{filt}'][j] * cat_gds['flux_auto'][j]) /cat_gds['flux_aper_1'][j])
    mag_gds.append(23.9-(2.5*np.log10(flux_gds[j])))
for j in range(len(cat_gdn)):
    flux_gdn.append((cat_gdn[f'{filt}'][j] * cat_gdn['flux_auto'][j]) /cat_gdn['flux_aper_1'][j])
    mag_gds.append(23.9-(2.5*np.log10(flux_gdn[j])))
for j in range(len(cat_cosmos_west)):
    flux_cosmos_west.append((cat_cosmos_west[f'{filt}'][j] * cat_cosmos_west['flux_auto'][j]) /cat_cosmos_west['flux_aper_1'][j])
    mag_cosmos_west.append(23.9-(2.5*np.log10(flux_cosmos_west[j])))
for j in range(len(cat_cosmos_east)):
    flux_cosmos_east.append((cat_cosmos_east[f'{filt}'][j] * cat_cosmos_east['flux_auto'][j]) /cat_cosmos_east['flux_aper_1'][j])
    mag_cosmos_west.append(23.9-(2.5*np.log10(flux_cosmos_east[j])))
for j in range(len(cat_udss)):
    flux_udss.append((cat_udss[f'{filt}'][j] * cat_udss['flux_auto'][j]) /cat_udss['flux_aper_1'][j])
    mag_udss.append(23.9-(2.5*np.log10(flux_udss[j])))
for j in range(len(cat_udsn)):
    flux_udsn.append((cat_udsn[f'{filt}'][j] * cat_udsn['flux_auto'][j]) /cat_udsn['flux_aper_1'][j])
    mag_udss.append(23.9-(2.5*np.log10(flux_udsn[j])))
for j in range(len(cat_ngdeep)):
    flux_ngdeep.append((cat_ngdeep[f'{filt}'][j] * cat_ngdeep['flux_auto'][j]) /cat_ngdeep['flux_aper_1'][j])
    mag_ngdeep.append(23.9-(2.5*np.log10(flux_ngdeep[j])))


In [ ]:
plt.hist(mag_ceers,histtype='step',bins=100,label='CEERS')
plt.hist(mag_gds,histtype='step',bins=100,label='GDS')
plt.hist(mag_cosmos_west,histtype='step',bins=100,label='PRIMER COSMOS')
plt.hist(mag_udss,histtype='step',bins=100,label='UDS')
plt.hist(mag_ngdeep,histtype='step',bins=100,label='NGDeep')
plt.xlabel('AB Magnitudes')
plt.title(f'AB Magnitudes for {filt}')
plt.legend()
plt.show()
